# 📗 FILE 2-A – Dataset API & Training Loop

## 🎯 Mục tiêu

Sau bài này bạn sẽ hiểu:
- **tf.data.Dataset** - API chuyên nghiệp cho data pipeline
- **Batch, Shuffle, Prefetch** - Tối ưu training speed
- **Custom training loop** - Kiểm soát hoàn toàn quá trình training
- **Validation loop** - Đánh giá model đúng cách
- **Overfitting vs Underfitting** - Nhận biết và khắc phục

---

## 📌 Tại sao học tf.data?

**Vấn đề với NumPy arrays:**
- Load toàn bộ data vào RAM → OOM với big datasets
- Không có optimization tự động
- Không streaming, không parallel loading

**tf.data giải quyết:**
- ✅ Streaming data (không cần load hết vào RAM)
- ✅ Auto prefetch & parallelization
- ✅ Transformation pipeline
- ✅ Production-ready

---

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time

print(f"TensorFlow version: {tf.__version__}")

---

## 1️⃣ tf.data.Dataset - Cơ bản

### 🔹 Tạo Dataset từ NumPy

In [ ]:
# Tạo data giả
X = np.random.randn(1000, 10).astype(np.float32)
y = np.random.randn(1000, 1).astype(np.float32)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# Tạo Dataset
dataset = tf.data.Dataset.from_tensor_slices((X, y))

print(f"\nDataset: {dataset}")
print(f"Element spec: {dataset.element_spec}")

In [ ]:
# Iterate through dataset
print("First 3 samples:")
for i, (x_sample, y_sample) in enumerate(dataset.take(3)):
    print(f"  Sample {i}: x.shape={x_sample.shape}, y.shape={y_sample.shape}")
    print(f"    x={x_sample.numpy()[:3]}...")  # First 3 features
    print(f"    y={y_sample.numpy()}")

### 🔹 Các cách tạo Dataset khác

In [ ]:
# Cách 1: from_tensor_slices (đã dùng)
ds1 = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5])
print("Method 1 - from_tensor_slices:")
print(list(ds1.as_numpy_iterator()))

# Cách 2: from_tensors (toàn bộ là 1 element)
ds2 = tf.data.Dataset.from_tensors([1, 2, 3, 4, 5])
print("\nMethod 2 - from_tensors:")
print(list(ds2.as_numpy_iterator()))

# Cách 3: range
ds3 = tf.data.Dataset.range(5)
print("\nMethod 3 - range:")
print(list(ds3.as_numpy_iterator()))

---

## 2️⃣ Dataset Transformations

### 🔹 map() - Transform từng element

In [ ]:
# Example: Square all numbers
dataset = tf.data.Dataset.range(5)
dataset_squared = dataset.map(lambda x: x ** 2)

print("Original:", list(dataset.as_numpy_iterator()))
print("Squared: ", list(dataset_squared.as_numpy_iterator()))

In [ ]:
# Example: Normalize images
def normalize(x, y):
    """Normalize x to [0, 1]"""
    x = x / 255.0
    return x, y

# Fake image data
images = np.random.randint(0, 256, size=(10, 28, 28, 1), dtype=np.uint8)
labels = np.random.randint(0, 10, size=(10,), dtype=np.int32)

dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset = dataset.map(normalize)

# Check
for x, y in dataset.take(1):
    print(f"Before normalize: 0-255")
    print(f"After normalize: min={x.numpy().min():.2f}, max={x.numpy().max():.2f}")

### 🔹 filter() - Lọc elements

In [ ]:
# Example: Keep only even numbers
dataset = tf.data.Dataset.range(10)
dataset_even = dataset.filter(lambda x: x % 2 == 0)

print("Original:", list(dataset.as_numpy_iterator()))
print("Even only:", list(dataset_even.as_numpy_iterator()))

---

## 3️⃣ Batch - Nhóm samples thành batches

### 🔹 Tại sao cần batch?

**Không batch (batch_size=1):**
```
For each sample:
    Forward pass → 1 sample
    Backward pass → Update weights
```
- ❌ Chậm (nhiều update)
- ❌ Gradient noisy
- ❌ Không tận dụng GPU

**Có batch (batch_size=32):**
```
For each batch of 32 samples:
    Forward pass → 32 samples parallel
    Backward pass → Average gradient
    Update weights
```
- ✅ Nhanh hơn
- ✅ Gradient stable hơn
- ✅ Tận dụng GPU tốt

---

In [ ]:
# Create dataset
X = np.random.randn(100, 10).astype(np.float32)
y = np.random.randn(100, 1).astype(np.float32)

dataset = tf.data.Dataset.from_tensor_slices((X, y))

# Batch
BATCH_SIZE = 32
dataset_batched = dataset.batch(BATCH_SIZE)

print(f"Original: {dataset.element_spec}")
print(f"Batched:  {dataset_batched.element_spec}")
print(f"\nNumber of batches: {len(list(dataset_batched))}")

In [ ]:
# Iterate through batches
for i, (x_batch, y_batch) in enumerate(dataset_batched.take(3)):
    print(f"Batch {i}: x.shape={x_batch.shape}, y.shape={y_batch.shape}")

### 🔹 drop_remainder - Xử lý batch cuối

In [ ]:
# 100 samples, batch_size=32
# → 3 batches of 32 + 1 batch of 4

dataset = tf.data.Dataset.from_tensor_slices(np.arange(100))

# Giữ batch cuối (mặc định)
batched_keep = dataset.batch(32, drop_remainder=False)
batch_sizes_keep = [len(list(batch.numpy())) for batch in batched_keep]
print(f"Keep last batch: {batch_sizes_keep}")

# Bỏ batch cuối
batched_drop = dataset.batch(32, drop_remainder=True)
batch_sizes_drop = [len(list(batch.numpy())) for batch in batched_drop]
print(f"Drop last batch: {batch_sizes_drop}")

---

## 4️⃣ Shuffle - Xáo trộn data

### 🔹 Tại sao cần shuffle?

**Không shuffle:**
```
Batch 1: [class_0, class_0, class_0, ...]
Batch 2: [class_1, class_1, class_1, ...]
```
- ❌ Model học theo thứ tự → bias
- ❌ Gradient không representative

**Có shuffle:**
```
Batch 1: [class_2, class_0, class_1, ...]
Batch 2: [class_1, class_0, class_2, ...]
```
- ✅ Model học balanced
- ✅ Gradient better estimate

---

In [ ]:
# Example: Ordered data
data = np.arange(20)
dataset = tf.data.Dataset.from_tensor_slices(data)

# No shuffle
print("Without shuffle:")
print(list(dataset.batch(5).take(2).as_numpy_iterator()))

# With shuffle
BUFFER_SIZE = 20
dataset_shuffled = dataset.shuffle(buffer_size=BUFFER_SIZE)
print("\nWith shuffle:")
print(list(dataset_shuffled.batch(5).take(2).as_numpy_iterator()))

### 🔹 buffer_size - Quan trọng!

**buffer_size** = số samples được load vào buffer để shuffle

```
buffer_size = 10:
  Load 10 samples → shuffle 10 samples → pick 1
  
buffer_size = 1000:
  Load 1000 samples → shuffle 1000 samples → pick 1
```

**Best practice:**
- Small dataset: `buffer_size = len(dataset)`
- Large dataset: `buffer_size = 10000` hoặc lớn hơn
- Quá nhỏ → shuffle không tốt
- Quá lớn → tốn RAM

---

In [ ]:
# Visualization: buffer_size effect
data = np.arange(100)
dataset = tf.data.Dataset.from_tensor_slices(data)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

buffer_sizes = [10, 50, 100]
for ax, buf_size in zip(axes, buffer_sizes):
    shuffled = dataset.shuffle(buf_size, seed=42)
    samples = list(shuffled.take(50).as_numpy_iterator())
    
    ax.scatter(range(len(samples)), samples, alpha=0.6, s=20)
    ax.plot(range(len(samples)), samples, alpha=0.3, linewidth=0.5)
    ax.set_title(f'buffer_size = {buf_size}')
    ax.set_xlabel('Index')
    ax.set_ylabel('Value')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Quan sát: buffer_size càng lớn → shuffle càng random")

---

## 5️⃣ Prefetch - Tối ưu performance

### 🔹 Vấn đề: GPU idle

**Không prefetch:**
```
Time:  |----GPU train----|----CPU load data----|----GPU train----|...
       ^                 ^
       GPU working        GPU IDLE (waiting for data)
```

**Có prefetch:**
```
Time:  |----GPU train----|----GPU train----|----GPU train----|...
       |----CPU load-----|----CPU load-----|----CPU load-----|
       ^
       CPU & GPU overlap
```

### 🔹 Cách dùng

In [ ]:
# Create dataset
X = np.random.randn(10000, 28, 28, 1).astype(np.float32)
y = np.random.randint(0, 10, size=(10000,), dtype=np.int32)

dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.batch(32)

# Without prefetch
print("Benchmark without prefetch...")
start = time.time()
for _ in dataset.take(100):
    pass
time_no_prefetch = time.time() - start

# With prefetch
dataset_prefetch = dataset.prefetch(tf.data.AUTOTUNE)
print("Benchmark with prefetch...")
start = time.time()
for _ in dataset_prefetch.take(100):
    pass
time_with_prefetch = time.time() - start

print(f"\nResults:")
print(f"  Without prefetch: {time_no_prefetch:.4f}s")
print(f"  With prefetch:    {time_with_prefetch:.4f}s")
print(f"  Speedup:          {time_no_prefetch/time_with_prefetch:.2f}x")

### 🔹 AUTOTUNE - Tự động điều chỉnh

```python
dataset.prefetch(tf.data.AUTOTUNE)  # Recommended!
```

- TensorFlow tự động chọn buffer size tối ưu
- Thích ứng với hardware

---

---

## 6️⃣ Complete Pipeline Pattern

### 🔹 Pattern chuẩn cho training

In [ ]:
def create_dataset(X, y, batch_size=32, shuffle=True, augment=False):
    """
    Create optimized tf.data pipeline
    
    Args:
        X: Features
        y: Labels
        batch_size: Batch size
        shuffle: Whether to shuffle
        augment: Whether to apply augmentation
    """
    # 1. Create dataset
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    
    # 2. Shuffle (if training)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(X))
    
    # 3. Batch
    dataset = dataset.batch(batch_size)
    
    # 4. Augmentation (if needed)
    if augment:
        dataset = dataset.map(
            lambda x, y: (x + tf.random.normal(tf.shape(x)) * 0.01, y),
            num_parallel_calls=tf.data.AUTOTUNE
        )
    
    # 5. Prefetch (ALWAYS!)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    
    return dataset

# Example usage
X_train = np.random.randn(1000, 10).astype(np.float32)
y_train = np.random.randn(1000, 1).astype(np.float32)

train_dataset = create_dataset(X_train, y_train, batch_size=32, shuffle=True)
val_dataset = create_dataset(X_train[:200], y_train[:200], batch_size=32, shuffle=False)

print(f"Train dataset: {train_dataset}")
print(f"Val dataset: {val_dataset}")

---

## 7️⃣ Custom Training Loop

### 🔹 Tại sao cần custom loop?

`model.fit()` tiện nhưng:
- ❌ Khó customize
- ❌ Không linh hoạt cho research
- ❌ Không control được từng step

**Custom loop:**
- ✅ Full control
- ✅ Custom metrics
- ✅ Advanced techniques (GAN, RL...)

### 🔹 Pattern cơ bản

In [ ]:
# Prepare data
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

X, y = make_regression(n_samples=1000, n_features=10, noise=10, random_state=42)
X = X.astype(np.float32)
y = y.reshape(-1, 1).astype(np.float32)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create datasets
train_dataset = create_dataset(X_train, y_train, batch_size=32, shuffle=True)
val_dataset = create_dataset(X_val, y_val, batch_size=32, shuffle=False)

print(f"Train samples: {len(X_train)}")
print(f"Val samples: {len(X_val)}")

In [ ]:
# Build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(10,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Optimizer & loss
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.MeanSquaredError()
metric = tf.keras.metrics.MeanAbsoluteError()

print("Model ready!")

In [ ]:
# Custom training loop
@tf.function  # Convert to graph for speed
def train_step(x, y):
    """Single training step"""
    with tf.GradientTape() as tape:
        # Forward pass
        predictions = model(x, training=True)
        loss = loss_fn(y, predictions)
    
    # Backward pass
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    # Update metric
    metric.update_state(y, predictions)
    
    return loss

@tf.function
def val_step(x, y):
    """Single validation step"""
    predictions = model(x, training=False)
    loss = loss_fn(y, predictions)
    metric.update_state(y, predictions)
    return loss

In [ ]:
# Training loop
EPOCHS = 20
history = {'train_loss': [], 'train_mae': [], 'val_loss': [], 'val_mae': []}

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
    
    # ============ TRAINING ============
    metric.reset_state()
    train_losses = []
    
    for x_batch, y_batch in train_dataset:
        loss = train_step(x_batch, y_batch)
        train_losses.append(loss)
    
    train_loss = np.mean(train_losses)
    train_mae = metric.result().numpy()
    
    # ============ VALIDATION ============
    metric.reset_state()
    val_losses = []
    
    for x_batch, y_batch in val_dataset:
        loss = val_step(x_batch, y_batch)
        val_losses.append(loss)
    
    val_loss = np.mean(val_losses)
    val_mae = metric.result().numpy()
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_mae'].append(train_mae)
    history['val_loss'].append(val_loss)
    history['val_mae'].append(val_mae)
    
    # Print progress
    print(f"  Train Loss: {train_loss:.4f}, MAE: {train_mae:.4f}")
    print(f"  Val Loss:   {val_loss:.4f}, MAE: {val_mae:.4f}")

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[0].plot(history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].set_title('Training & Validation Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(history['train_mae'], label='Train MAE', linewidth=2)
axes[1].plot(history['val_mae'], label='Val MAE', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].set_title('Training & Validation MAE')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 8️⃣ Overfitting vs Underfitting

### 🔹 Định nghĩa

**Underfitting:**
- Model **quá đơn giản**
- Không học được patterns
- Train loss cao, Val loss cao

**Good fit:**
- Model vừa đủ
- Học được patterns, generalize tốt
- Train loss thấp, Val loss thấp

**Overfitting:**
- Model **quá phức tạp**
- Học thuộc training data
- Train loss thấp, Val loss cao

---

In [ ]:
# Generate data with noise
np.random.seed(42)
X_demo = np.linspace(0, 10, 100).reshape(-1, 1).astype(np.float32)
y_demo = (np.sin(X_demo) + np.random.randn(100, 1) * 0.3).astype(np.float32)

X_train_demo, X_val_demo, y_train_demo, y_val_demo = train_test_split(
    X_demo, y_demo, test_size=0.2, random_state=42
)

In [ ]:
# Helper function to train and plot
def train_and_evaluate(model, X_train, y_train, X_val, y_val, epochs=100, title=""):
    """Train model and return history"""
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=16,
        verbose=0
    )
    return history

# 1. UNDERFITTING - Model quá đơn giản
model_underfit = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=(1,))  # Only 1 neuron!
])
model_underfit.compile(optimizer='adam', loss='mse')

# 2. GOOD FIT - Model vừa đủ
model_goodfit = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(1,)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)
])
model_goodfit.compile(optimizer='adam', loss='mse')

# 3. OVERFITTING - Model quá phức tạp + train quá lâu
model_overfit = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(1,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])
model_overfit.compile(optimizer='adam', loss='mse')

print("Training models...")
hist_underfit = train_and_evaluate(model_underfit, X_train_demo, y_train_demo, 
                                   X_val_demo, y_val_demo, epochs=100)
hist_goodfit = train_and_evaluate(model_goodfit, X_train_demo, y_train_demo,
                                  X_val_demo, y_val_demo, epochs=100)
hist_overfit = train_and_evaluate(model_overfit, X_train_demo, y_train_demo,
                                  X_val_demo, y_val_demo, epochs=500)  # Train longer
print("Done!")

In [ ]:
# Visualize learning curves
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

histories = [
    (hist_underfit, 'Underfitting'),
    (hist_goodfit, 'Good Fit'),
    (hist_overfit, 'Overfitting')
]

for ax, (hist, title) in zip(axes, histories):
    ax.plot(hist.history['loss'], label='Train Loss', linewidth=2)
    ax.plot(hist.history['val_loss'], label='Val Loss', linewidth=2)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.set_title(title)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Add interpretation
    final_train = hist.history['loss'][-1]
    final_val = hist.history['val_loss'][-1]
    ax.text(0.5, 0.95, f'Train: {final_train:.4f}\nVal: {final_val:.4f}',
            transform=ax.transAxes, ha='center', va='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print("\nNhận xét:")
print("1. UNDERFITTING: Train & Val loss đều cao → Model chưa đủ mạnh")
print("2. GOOD FIT: Train & Val loss đều thấp và gần nhau → Perfect!")
print("3. OVERFITTING: Train loss thấp nhưng Val loss cao → Model học thuộc")

### 🔹 Cách khắc phục

**Underfitting:**
- ✅ Tăng model capacity (more layers/neurons)
- ✅ Train lâu hơn
- ✅ Thêm features

**Overfitting:**
- ✅ Regularization (L1, L2, Dropout) → Sẽ học ở File 2-B
- ✅ Early stopping
- ✅ More data
- ✅ Giảm model complexity

---

---

## 9️⃣ Best Practices Summary

### ✅ tf.data Pipeline Pattern

```python
dataset = (
    tf.data.Dataset.from_tensor_slices((X, y))
    .shuffle(buffer_size=len(X))  # Training only
    .batch(batch_size)
    .map(preprocess_fn, num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)  # ALWAYS!
)
```

### ✅ Custom Training Loop Pattern

```python
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        predictions = model(x, training=True)
        loss = loss_fn(y, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

for epoch in range(epochs):
    for x_batch, y_batch in train_dataset:
        loss = train_step(x_batch, y_batch)
```

### ✅ Monitoring Checklist

- [ ] Train loss giảm?
- [ ] Val loss giảm?
- [ ] Train vs Val loss có gap lớn không? (overfitting)
- [ ] Cả 2 đều cao? (underfitting)

---

---

## 🔟 Exercises

### 📝 Exercise 1: Dataset Pipeline

Tạo optimized dataset pipeline cho MNIST:
- Load MNIST
- Normalize to [0, 1]
- Shuffle, batch, prefetch
- Augmentation: random rotation ±10 degrees

In [ ]:
# YOUR CODE HERE
# TODO: Load MNIST
# TODO: Create pipeline with augmentation

### 📝 Exercise 2: Custom Training with Metrics

Implement custom training loop với:
- Custom metric (F1-score for classification)
- Progress bar (tqdm)
- Save best model based on val metric

In [ ]:
# YOUR CODE HERE
# TODO: Implement custom loop with F1-score

### 📝 Exercise 3: Detect Overfitting

Given training history, viết function detect overfitting:
- Return True nếu val_loss tăng 3 epochs liên tiếp
- Return False otherwise

In [ ]:
# YOUR CODE HERE
def detect_overfitting(history, patience=3):
    """
    Detect overfitting from training history
    
    Args:
        history: Dictionary with 'val_loss' key
        patience: Number of epochs to check
    
    Returns:
        bool: True if overfitting detected
    """
    # TODO: Implement
    pass

# Test
history_good = {'val_loss': [1.0, 0.8, 0.6, 0.5, 0.4]}
history_bad = {'val_loss': [1.0, 0.8, 0.9, 1.0, 1.1]}

print(detect_overfitting(history_good))  # Should be False
print(detect_overfitting(history_bad))   # Should be True

---

## 🎯 Tóm tắt

### ✅ Đã học

1. **tf.data.Dataset** - Professional data pipeline
2. **Transformations** - map, filter, batch, shuffle
3. **Prefetch** - Optimize GPU utilization
4. **Custom training loop** - Full control
5. **Overfitting vs Underfitting** - Recognize and fix

### 🎓 Key Takeaways

**Always use this pattern:**
```python
dataset = (
    tf.data.Dataset.from_tensor_slices(data)
    .shuffle(buffer_size)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)
```

**Monitor training:**
- Train loss ↓, Val loss ↓ → Good!
- Train loss ↓, Val loss ↑ → Overfitting!
- Both high → Underfitting!

### 📚 Next Steps

- **File 2-B**: Optimizers, Activations & Regularization
- **File 2-C**: CNN & Callbacks

---

## 📖 References

- [tf.data Guide](https://www.tensorflow.org/guide/data)
- [tf.data Performance](https://www.tensorflow.org/guide/data_performance)
- [Custom Training](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch)

---

**Chúc bạn học tốt! 🚀**